In [7]:
# cmd shift 5 for area, this code for x_y
import pyautogui
import time

print("🖱️  Mouse position (1/sec) - Ctrl+C to quit")
try:
    while True:
        x, y = pyautogui.position()
        print(f"\rX: {x:4d} Y: {y:4d}", end='', flush=True)
        time.sleep(1)
except KeyboardInterrupt:
    print("\n✅ Stopped.")


🖱️  Mouse position (1/sec) - Ctrl+C to quit
X:  876 Y:  183
✅ Stopped.


In [ ]:
import pyautogui
from PIL import Image, ImageEnhance, ImageFilter, ImageOps
from gradio_client import Client, handle_file
import pandas as pd
from bs4 import BeautifulSoup
import time
from datetime import datetime
import os


# CONFIGURATION
TABLE_X, TABLE_Y, TABLE_WIDTH, TABLE_HEIGHT = 45, 235, 287, 220
INTERVAL_SECONDS = 2.0
CSV_FILE = "fidelity_orderbook_log.csv"
CLEAN_CSV_FILE = "fidelity_orderbook_clean.csv"
CLEAN_EVERY_N_ITERATIONS = 1  # Save clean CSV every 1 iterations


# REUSE CLIENT (critical for speed)
client = Client("deepdoctection/deepdoctection")


print(f"🚀 Starting order book monitor (every {INTERVAL_SECONDS}s)")
print(f"Table region: ({TABLE_X}, {TABLE_Y}, {TABLE_WIDTH}, {TABLE_HEIGHT})")
print(f"Clean CSV will update every {CLEAN_EVERY_N_ITERATIONS} iterations")


iteration = 0
clean_iteration_counter = 0


try:
    while True:
        start_time = time.time()
        iteration += 1
        clean_iteration_counter += 1


        try:
            # 1. CAPTURE SCREENSHOT
            screenshot = pyautogui.screenshot(region=(TABLE_X, TABLE_Y, TABLE_WIDTH, TABLE_HEIGHT))


            # 2. PREPROCESS IMAGE
            img = screenshot.convert("RGB")
            gray = ImageEnhance.Contrast(img.convert("L")).enhance(1.2)
            gray = gray.filter(ImageFilter.SHARPEN)
            inverted = ImageOps.invert(gray)
            binary = inverted.point(lambda p: 0 if p > 140 else 255)
            binary.save("processed_table_region.png")


            # 3. OCR VIA DEEPDOCTECTION
            result = client.predict(
                img=handle_file("processed_table_region.png"),
                pdf=None,
                max_datapoints=1,
                api_name="/analyze_image",
            )


            # 4. PARSE TABLE AND SAVE RAW DATA
            if isinstance(result, tuple) and len(result) >= 4:
                html_table = result[3]
                soup = BeautifulSoup(html_table, "html.parser")
                table = soup.find("table")
                df = pd.read_html(str(table))[0]


                if df.columns.isnull().any():
                    df.columns = [f"col_{i}" if pd.isna(c) else str(c).strip() for i, c in enumerate(df.columns)]
                else:
                    df.columns = [str(c).strip() for c in df.columns]


                flat_values = df.to_numpy().ravel()
                col_labels = []
                for i in range(len(df)):
                    for col in df.columns:
                        col_labels.append(f"{col}_{i+1}")


                if len(col_labels) != len(flat_values):
                    min_len = min(len(col_labels), len(flat_values))
                    col_labels = col_labels[:min_len]
                    flat_values = flat_values[:min_len]


                row_data = [datetime.now().strftime("%Y-%m-%d %H:%M:%S")] + list(flat_values)
                col_names = ["timestamp"] + col_labels


                single_row = pd.DataFrame([row_data], columns=col_names)


                header_needed = not os.path.exists(CSV_FILE)
                single_row.to_csv(CSV_FILE, index=False, mode="a", header=header_needed)


                print(f"[{iteration:3d}] {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - Raw data saved")


            else:
                print(f"[{iteration:3d}] No table detected")


        except Exception as e:
            print(f"[{iteration:3d}] Error: {e}")
            continue


        # ===============================
        # PERIODIC CLEANUP AND SAVE (FIXED)
        # ===============================
        if clean_iteration_counter >= CLEAN_EVERY_N_ITERATIONS and os.path.exists(CSV_FILE):
            try:
                print(f"[{iteration:3d}] Running cleanup (iteration {clean_iteration_counter})...")
                
                # FIXED: Robust CSV reading that skips malformed rows
                df = pd.read_csv(
                    CSV_FILE,
                    on_bad_lines="skip",  # Skip rows with wrong # of columns
                    engine="python"       # More tolerant parser
                )
                
                BID_HEADERS = ['exch_b', 'size_b', 'bid', 'ask', 'size_a', 'exch_a']


                # Keep timestamp + data columns (skip first 6 junk header columns)
                data_columns = df.columns[6:]
                df_data = df[['timestamp'] + list(data_columns)].copy()


                # Drop first junk column and shift left by one
                first_junk_col = data_columns[0]
                df_data = df_data.drop(columns=[first_junk_col])


                # Create proper alternating column names
                col_names = ["timestamp"]
                level = 1
                for i in range(len(df_data.columns) - 1):  # Adjust for dropped column
                    base_header = BID_HEADERS[i % 6]
                    col_names.append(f"{base_header}_{level}")
                    if (i + 1) % 6 == 0:
                        level += 1


                df_data.columns = col_names
                df_data.to_csv(CLEAN_CSV_FILE, index=False)


                print(f"✅ Clean CSV updated → {CLEAN_CSV_FILE} ({len(df_data)} rows)")
                clean_iteration_counter = 0  # Reset counter
                
            except Exception as cleanup_error:
                print(f"[{iteration:3d}] Cleanup error: {cleanup_error}")


        elapsed = time.time() - start_time
        sleep_time = max(0, INTERVAL_SECONDS - elapsed)
        if sleep_time > 0:
            time.sleep(sleep_time)


except KeyboardInterrupt:
    print("\n🛑 Stopped by user — final cleanup...")


# FINAL CLEANUP (runs on Ctrl+C) - FIXED
if os.path.exists(CSV_FILE):
    try:
        print("Running final cleanup...")
        # FIXED: Robust CSV reading
        df = pd.read_csv(
            CSV_FILE,
            on_bad_lines="skip",
            engine="python"
        )
        
        BID_HEADERS = ['exch_b', 'size_b', 'bid', 'ask', 'size_a', 'exch_a']


        data_columns = df.columns[6:]
        df_data = df[['timestamp'] + list(data_columns)].copy()
        first_junk_col = data_columns[0]
        df_data = df_data.drop(columns=[first_junk_col])


        col_names = ["timestamp"]
        level = 1
        for i in range(len(df_data.columns) - 1):
            base_header = BID_HEADERS[i % 6]
            col_names.append(f"{base_header}_{level}")
            if (i + 1) % 6 == 0:
                level += 1


        df_data.columns = col_names
        df_data.to_csv(CLEAN_CSV_FILE, index=False)


        print(f"✅ Final clean CSV saved → {CLEAN_CSV_FILE} ({len(df_data)} rows)")
        print("First 7 columns:")
        print(list(df_data.columns[:7]))
        print("\nSample preview:")
        print(df_data[['timestamp', 'exch_b_1', 'size_b_1', 'bid_1', 'ask_1', 'size_a_1', 'exch_a_1']].head(2))
    except Exception as final_error:
        print(f"Final cleanup error: {final_error}")
else:
    print("⚠️ No raw CSV file found.")


Loaded as API: https://deepdoctection-deepdoctection.hf.space ✔
🚀 Starting order book monitor (every 2.0s)
Table region: (45, 235, 287, 220)
Clean CSV will update every 1 iterations


/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_68127/1535705702.py:67: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


[  1] 2025-12-08 14:38:02 - Raw data saved
[  1] Running cleanup (iteration 1)...
✅ Clean CSV updated → fidelity_orderbook_clean.csv (1232 rows)


/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_68127/1535705702.py:67: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


[  2] 2025-12-08 14:38:11 - Raw data saved
[  2] Running cleanup (iteration 1)...
✅ Clean CSV updated → fidelity_orderbook_clean.csv (1233 rows)


...

[ 43] 2025-12-08 14:48:00 - Raw data saved
[ 43] Running cleanup (iteration 1)...
✅ Clean CSV updated → fidelity_orderbook_clean.csv (1278 rows)


/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_68127/1535705702.py:67: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


[ 44] 2025-12-08 14:48:16 - Raw data saved
[ 44] Running cleanup (iteration 1)...
✅ Clean CSV updated → fidelity_orderbook_clean.csv (1279 rows)


/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_68127/1535705702.py:67: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


[ 45] 2025-12-08 14:48:33 - Raw data saved
[ 45] Running cleanup (iteration 1)...
✅ Clean CSV updated → fidelity_orderbook_clean.csv (1280 rows)


In [15]:
# Show ALL rows of df_data (temporarily override pandas display limits)
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', None):
    print(df_data)

# Or save to file for easier viewing
df_data.to_csv("full_orderbook_view.csv", index=False)
print(f"\n💾 Full dataset also saved to full_orderbook_view.csv ({len(df_data)} rows, {len(df_data.columns)} columns)")


               timestamp exch_b_1     size_b_1   bid_1                  ask_1  \
0    2025-12-06 09:59:01     ARCX            1  454.61                 454.64   
1    2025-12-06 09:59:10     ARCX            1  454.61                 454.64   
2    2025-12-06 09:59:18     ARCX            1  454.61                 454.64   
3    2025-12-06 09:59:26     ARCX            1  454.61                 454.64   
4    2025-12-06 09:59:34     ARCX            1  454.61                 454.64   
5    2025-12-06 09:59:42     ARCX            1  454.61                 454.64   
6    2025-12-06 09:59:51     ARCX            1  454.61                 454.64   
7    2025-12-06 09:59:59     ARCX            1  454.61                 454.64   
8    2025-12-06 10:00:07     ARCX            1  454.61                 454.64   
9    2025-12-06 10:00:15     ARCX            1  454.61                 454.64   
10   2025-12-06 10:00:23     ARCX            1  454.61                 454.64   
11   2025-12-06 10:00:31    